# 6장 데이터 로딩과 저장, 파일 형식

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
pd.options.display.max_rows = 10
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [ ]:
#!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv") # 데이터프레임으로 반환
df.head(2)

    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [ ]:
#!cat examples/ex2.csv
names = ["a", "b", "c", "d", "message"]
data = pd.read_csv("examples/ex2.csv", names=names)
data['message'] # Series

    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [ ]:
# !cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed.columns

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [ ]:
#!cat examples/ex3.txt

result = pd.read_csv("examples/ex3.txt",sep="\s+") #\s+ -> 스페이스바가 0개 이상
result

    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [ ]:
# !cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

    6) **누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력** 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [ ]:
# !cat examples/ex5.csv

result = pd.read_csv("examples/ex5.csv")
result

# 1) 누락된 값을 확인 : isna(), isnull(): 결측치 True, isnot(), notNa(): 결측치가 아닌 것 True
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
result
result.isna()

    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 읽을 파일의 크기

In [ ]:
# result = pd.read_csv("examples/ex6.csv")
# result
result = pd.read_csv("examples/ex6.csv", nrows=5)
result

### [예제]  ex6.csv 파일의 "key" 컬럼의 값들에 대한 빈도수을 계산하기

In [ ]:
#!cat examples/ex6.csv
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
chunker
tot = pd.Series([], dtype='int64')

# "key" 컬럼의 값들에 대한 누적의 합
for piece in chunker:# 10번 반복 수행
    # print(piece["key"].value_counts())
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot
#값들을 정렬
tot = tot.sort_values(ascending=False)
tot
tot[:10]

In [ ]:
# 시리즈 간 덧셈 연산
from pandas import Series
tot = Series([], dtype= 'int64')
tot1 = Series([1,2,3])
#tot + tot1
# tot.add(tot1)
# 누적의 합
for i in range(3):
    tot = tot.add(tot1, fill_value = 0)
tot

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [ ]:
# csv 파일로 저장
# 1. 데이터프레임 준비
data = pd.read_csv("examples/ex5.csv") # 원본에 대한 데이터프레임
data
data1 = data[['a','b','d']] # 컬럼 인덱싱 이용 데이터 추출
data1
# 2. 데이터프레임을 파일로 저장
data1.to_csv("examples/out.csv")

In [ ]:
data.to_csv("examples/out.csv", sep="|") # 구분자 사용
data.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
data.to_csv("examples/out.csv", index=False, header=False)
data.to_csv("examples/out.csv", index=False, columns=["a", "b", "c"])


# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저장
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [ ]:
import pandas as pd

# 1. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)

# 2. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# 3. JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)

# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서의 테이블 데이터 읽어오기
    * 연도별로 부도 은행의 개수을 계산하시오. 

In [ ]:
import pandas as pd
#tables = pd.read_html("examples/fdic_failed_bank_list.html") 
#1. 정적웹페이지 데이터 읽어오기
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
type(tables)
len(tables)
failures = tables[0] #인덱싱
failures
type(failures)
failures.head()

In [ ]:
# 2. 연도별로 부도 은행의 개수 : DataFrame 가공
# 1) 컬럼명 확인 /수정하기 /확인
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert', 'AcquiringInstitutionAI', \
                    'ClosingDate', 'FundFund']
failures.head(3)

In [ ]:
# 2) 데이터 변환 : 문자열 object -> datetime
failures["ClosingDate"] # 컬럼 인덱싱을 이용해서 시리즈로 추출
close_timestamps = pd.to_datetime(failures["ClosingDate"]) 
close_timestamps
# 3) 년도별 데이터 개수 추출
close_timestamps.dt.year
close_timestamps.dt.year.value_counts()

### [문제] : 네이버 주식 데이터을 스크래핑하여 csv파일로 저장하기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [ ]:
!pip install html5lib  lxml

In [ ]:
import pandas as pd
# 1) 웹페이지 읽어오기 
# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
df_list
type(df_list)
df_list[1] # 인덱싱
# 데이터프레임 확인
for df in df_list:
    print("\n",df)


In [ ]:
# 2) 데이터프레임 가공 : csv파일로 저장하기
df = df_list[1]
type(df)
df.head(10)
# (1) 데이터프레임 탐색 :  종목명 컬럼명으로 인덱싱 - NaN 처리
df['종목명']
# (2) 조건색인
cond = df['종목명'].notnull() #불리언 시리즈
data = df[cond] # 불리언색인을 이용하여 필터링
data
data.head(10)
#3) 데이터프레임 저장
data.to_csv("examples/Naver_kospi.csv")

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 패키지 requests을 활용하여 웹페이지 json 데이터을 데이터프레임으로 변환하기

In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd
#1. 타겟 웹사이트 접속
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.status_code
# 2. 웹페이지 콘텐츠 읽어오기
resp.text # 문자열로 반환
#pd.read_html(url)
# 3. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
print(data[0].keys()) # 키 정보 출력
# data[0]['number']

#4. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()


### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [ ]:
import requests
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

# params = {
#     'pageNo' : 1,
#     'rangeType' : 'ALL',
#     'orderBy' : 'sim',
#     'keyword' : '파이썬'
# }

# response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)
response = requests.get(url)
print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
  * get_attribute()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup
#타겟 웹사이트 지정
url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

# 웹페이지 로딩
response = requests.get(url)

if response.status_code == 200:    
    # beautifulsoup 객체로 변환
    soup = BeautifulSoup(response.text, 'html.parser')

    # 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # print(title)
    # print(title.get_text())
    # print(title.get_attribute_list("href"))

    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    # s_content > div.section > ul > li:nth-child(2) > dl > dt > a
    # s_content > div.section > ul > li:nth-child(3) > dl > dt > a
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    # print(titles)
    for title in titles:
        print("\n",title.get_text())
        print("\n",title.get_attribute_list("href"))
else : 
    print(response.status_code)



## 3.4 웹 스크래핑 - Selenium을 이용하여 크롤러( 6주차)
*  https://selenium-python.readthedocs.io/
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제]Selenium을 이용하여 자동으로 타겟 웹 사이트 실행하기
* 타겟 사이트 : https://www.google.com

In [ ]:
!pip install selenium

In [5]:
from selenium import webdriver
# 1. Chrome 브라우저 실행
driver = webdriver.Chrome() # 크롬 드라이버 실행파일의 위치 - 작업디렉터리
# 2. 타겟 웹페이지 로딩
url = "https://www.google.com"
driver.get(url)
# 3. 타겟 브라우저 종료
driver.quit()

### [문제] Selenium을 활용하여 타겟 웹사이트 자동으로 열기/ 쿼리로 검색하기/ 검색 결과의 제목과 url을 자동으로 추출하여 저장하는 web crawler 만들기
    * 사전 준비 작업 : 타겟 웹사이트을 구글 개발도구을 이용하여 탐색하기
    * 타겟 사이트: https://www.google.com
    * from selenium.webdriver.common.by import By
    *from selenium.webdriver.common.keys import Keys
    * driver.find_element()
    * send_keys()

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time # python 내장함수

# 1. Chrome 브라우저 실행
driver = webdriver.Chrome()

# 2. 타겟 Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 3. 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(2)

# 4. 검색 기능 - 검색창-> 검색어 입력 -> 엔터 키로 클릭 처리
try:
    # 4-1 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q") 

    # 4-2 검색어 입력
    search_query = input("검색키 입력: ")
    search_box.send_keys(search_query) 

    # 4-3 Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 5. 검색에 관련된  결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")
    # print(search_results[0])
        
    # 5-1 첫 번째 제목과 링크 url 추출
    # title = search_results[0].find_element(By.CSS_SELECTOR, "h3").text
    # url = search_results[0].find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    # print("제목:", title)
    # print("URL:", url)

    # 5-2 검색 결과의 모든 제목과 링크 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
    
except Exception as e:
    print("오류가 발생:", e)
    exit()

finally:
    # 브라우저 창 닫기
    driver.quit()

검색키 입력:  총선 결과


제목: 22대 총선 개표 결과 보기
URL: https://www.bbc.com/korean/articles/c06045ln53lo
제목: [총선] 민주당 170석 이상 압승·국민의힘 참패 전망
URL: https://news.kbs.co.kr/news/view.do?ncd=7936696
제목: 제22대 국회의원 선거/정당별 결과/국민의힘
URL: https://namu.wiki/w/%EC%A0%9C22%EB%8C%80%20%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90%20%EC%84%A0%EA%B1%B0/%EC%A0%95%EB%8B%B9%EB%B3%84%20%EA%B2%B0%EA%B3%BC/%EA%B5%AD%EB%AF%BC%EC%9D%98%ED%9E%98
제목: 22대 총선 결과 보건의료인 후보 다수 당선
URL: http://www.bosa.co.kr/news/articleView.html?idxno=2220380
제목: 22대 총선 결과…민주 '압도적 과반'·與 '3연패' | 뉴스A 라이브
URL: https://www.youtube.com/watch?v=AGFg1n3QqNM
제목: [다시보기] 22대 총선 결과…민주 '압도적 과반'·與 '3연패' 2024 ...
URL: https://www.youtube.com/watch?v=oL969Le0F40
제목: 대한민국 제21대 국회의원 선거
URL: https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%EC%A0%9C21%EB%8C%80_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0
제목: [그래픽] 4.10 총선 서울지역 당선결과
URL: https://www.newspim.com/news/view/20240411000149


### [문제] Selenium을 이용하여 네이버 증권 웹페이지 열기/ 삼성전자 일별 시세 HTML가져오기/ 10페이지 내역 테이블 데이터 찾기/검색 결과를 데이터프레임으로 변환/저장하기
* 타겟 사이트: https://finance.naver.com/item/sise_day.naver?code=005930&page=

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [20]:
#1. 크롬 드라이버 실행
driver= webdriver.Chrome()

# 2페이지 로딩 시간
time.sleep(2)

# 3. 크롤링 사이트 : 삼성전자의 일별 시세 페이지 URL 찾기
samsung_url ="https://finance.naver.com/item/sise_day.naver?code=005930&page=" #base url

# 4. 첫 페이지부터 10 페이지까지 테이블 담기
samsung_daily_kospi = [] # 리스트 선언

for num in range(1,11):
    full_url = samsung_url + str(num)
    # 4.1 페이지별 요청
    driver.get(full_url)
        
    try:
        # 4.2 페이지의 HTML 가져오기
        html = driver.page_source
        df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
        # print(type(df))
        # print(df)
        # 4.3 결측치 처리 - 조건 색인을 이용한 불리언 인덱싱
        ## 조건 색인 인덱싱
        cond = df[0]['날짜'].notnull()
        samsung_day = df[0][cond]
        ## 리스트에 담기
        samsung_daily_kospi.append(samsung_day)
                
    except Exception as e:
        print("Network Error",e)
    
    # finally:
    #     driver.quit()

# 5. 데이터프레임 합치기
print(samsung_daily_kospi[:3])
samsung_daily_kospi_all = pd.concat(samsung_daily_kospi, ignore_index=True)
# samsung_daily_kospi_all.head(20)
# 6. csv 파일로 저장
samsung_daily_kospi_all.to_csv("samsung_daily_kospi_2024_04_11.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_9344\1237361889.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이블 구조 -> 데이터프레임


MaxRetryError: HTTPConnectionPool(host='localhost', port=13144): Max retries exceeded with url: /session/bf9b25c1a573280823461b3a0c5a0d34/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022510A66200>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))